# Ring Network 

## Imports

In [ ]:
# Standard Library
from collections import OrderedDict
from copy import copy

# Photontorch
import photontorch as pt

# Python
import tqdm
import torch
import numpy as np
import matplotlib.pyplot as plt

# numpy settings
np.random.seed(6) # seed for random numbers
np.set_printoptions(precision=2, suppress=True) # show less numbers while printing numpy arrays

## Simulation Environments

In [ ]:
env = pt.Environment(
    wls=np.linspace(1.5e-6,1.6e-6,10001),
    frequency_domain=True
)
pt.set_environment(env)

## Ring 'Molecule' Network

A ring 'molecule' is a network of ring resonators, where the ring resonators are the 'atoms'. The simplest ring 'molecule' consists of a single 'atom' (ring resonator):

### Single Ring

In [ ]:
ring = pt.RingMolecule(
    map='o',
    rings={
        'o':pt.Waveguide(length=2*np.pi*10e-6, neff=2.86),
    },
)

Creating a ring molecule always has the same form. You specify a string `map`, signifying the locations of the rings. Each character in the ring corresponds to a different kind of ring. In our case, the map contains a single character, since we only created a single ring.
To know the properties of each ring, the `RingMolecule` constructor needs a second argument: `rings`, where the waveguide making up the ring is specified.

This single ring example is not very useful, because it has no open ports to connect to. For this, you can use the `@` character in the string map. This specifies where to 'cut open' the ring, resulting in two new ports:

In [ ]:
wg = pt.RingMolecule(
    map='@o',
    rings={
        'o':pt.Waveguide(length=2*np.pi*10e-6, neff=2.86),
    }
)

note that, because the ring is 'cut open' it will now just act as a normal waveguide! We will prove this, by terminating the wg with a source and a detector.

In [ ]:
nw = wg.terminate([pt.Source('inp'), pt.Detector('outp')])
detected = nw(source=1, power=False)[:,0,:,0,0].detach().numpy()
detected_power = detected[0]**2 + detected[1]**2
detected_phase = np.arctan2(detected[1], detected[0])

# visualize
p1, = nw.plot(detected_power)
plt.ylim(0,1.1)
plt.gca().twinx()
p2, = nw.plot(detected_phase, color='C1', alpha=0.5)
plt.ylabel('phase [rad]')
plt.figlegend([p1,p2], ['power','phase'], ncol=2)
plt.show()

## All Pass Filter

So how do you make an all-pass filter then? Well, you use two rings, and cut one open:

In [ ]:
allpass = pt.RingMolecule(
    map='@oO',
    rings={
        'O':pt.Waveguide(2*np.pi*10e-6, loss=800, neff=2.86),
        'o':pt.Waveguide(0, loss=0, neff=0),
    },
    coupling={
        'oO':0.2
    }
).terminate([pt.Source('inp'), pt.Detector('pass')])

Note that you need another argument during the costruction of the allpass filter: `coupling`. This argument should be a dictionary specifying all the different couplings between the different ring types.

In [ ]:
detected = allpass(source=1, power=False)[:,0,:,0,0].detach().numpy()
detected_power = detected[0]**2 + detected[1]**2
detected_phase = np.arctan2(detected[1], detected[0])

# visualize
p1, = allpass.plot(detected_power)
plt.ylim(0,1.1)
plt.gca().twinx()
p2, = allpass.plot(detected_phase, color='C1', alpha=0.5)
plt.ylabel('phase [rad]')
plt.figlegend([p1,p2], ['power','phase'], ncol=2)
plt.show()

## Multiring All Pass Filter

In [ ]:
allpass = pt.RingMolecule(
    map='@oOOOOO',
    rings={
        'O':pt.Waveguide(2*np.pi*10e-6, loss=800, neff=2.86),
        'o':pt.Waveguide(0, loss=0, neff=0),
    },
    coupling={
        'oO':0.005,
        'OO':0.005,
    }
).terminate([pt.Source('inp'), pt.Detector('pass')])

In [ ]:
detected = allpass(source=1, power=False)[:,0,:,0,0].detach().numpy()
detected_power = detected[0]**2 + detected[1]**2
detected_phase = np.arctan2(detected[1], detected[0])

# visualize
p1, = allpass.plot(detected_power)
plt.ylim(0.7,1.1)
plt.gca().twinx()
p2, = allpass.plot(detected_phase, color='C1', alpha=0.5)
plt.ylabel('phase [rad]')
plt.figlegend([p1,p2], ['power','phase'], ncol=2)
plt.show()

## Add Drop Filter

Similarly, we can create an Add Drop Filter

In [ ]:
adddrop = pt.RingMolecule(
    map='@oOo@',
    rings={
        'O':pt.Waveguide(length=2*np.pi*10e-6, loss=800, neff=2.86),
        'o':pt.Waveguide(length=0),
    },
    coupling=0.2,
).terminate([pt.Source('inp'),  pt.Detector('pass'), pt.Detector('drop'), pt.Detector('add')])

In [ ]:
detected = adddrop(source=1, power=False)[:,0,:,0,0].detach().numpy()
detected_power = detected[0]**2 + detected[1]**2
detected_phase = np.arctan2(detected[1], detected[0])

# visualize
p1, = adddrop.plot(detected_power)
plt.ylim(0,1.1)
plt.gca().twinx()
p2, = adddrop.plot(detected_phase, color='C1', alpha=0.5)
plt.ylabel('phase [rad]')
plt.figlegend([p1,p2], ['power','phase'], ncol=2)
plt.show()

## Ring-Loaded Add-Drop Filter

An add drop filter with extra rings in the following pattern:


```
 O
OoO
 O
```

with `o` a defect ring with smaller radius than `O`

In [ ]:
period=20e-6
width=0.45e-6
gap=0.26e-6
nw = pt.RingMolecule(
    map='''...O...
           @xOoOx@
           ...O...''',
    rings={
        'O':pt.Waveguide(length=np.pi*(period-gap-width), loss=1000, neff=2.4, ng=4.3),
        'o':pt.Waveguide(length=np.pi*(period-gap-width-0.01e-6), loss=900, neff=2.43, ng=4.2),
        'x':pt.Waveguide(length=0),
    },
    coupling={
        'xO':0.05,
        'OO':0.05,
        'oO':0.04,
        'oo':0.038,
    },
).terminate([pt.Source('inp'), pt.Detector('pass'), pt.Detector('drop'), pt.Detector('add')])


# visualize
detected = nw(source=1)
nw.plot(torch.sqrt(detected))
plt.show()